In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from scipy import sparse
import sys
from sklearn.utils import shuffle
import joblib
sys.path.append("../")
from helper_functions import *

In [2]:
x_train = sparse.load_npz("Data/xtrain.npz")
x_test = sparse.load_npz("Data/xtest.npz")
y_train = np.load("Data/ytrain.npy")
y_test = np.load("Data/ytest.npy")

In [3]:
x_train = x_train.toarray()
x_test = x_test.toarray()
x = np.concatenate([x_train, x_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)
x = sparse.csr_matrix(x)

In [4]:
x,y = shuffle(x,y, random_state=1)

In [7]:
log_reg = LogisticRegression()
lgbm = LGBMClassifier()
gboost = GradientBoostingClassifier()
cat = CatBoostClassifier()

In [6]:
models = {
    'LogisticRegression': log_reg,
    'LGBMClassifier': lgbm,
    'GradientBoostingClassifier': gboost,
    'CatBoostClassifier': cat
}

In [7]:
BestParam_search(models, x, y, 2)

Starting Grid Search for Model: LogisticRegression


## The Result for Logistic Regression and LGBM classifier were as follows:
<font size ='5'>
Starting Grid Search for Model: LogisticRegression  <br>                            
Search Finished                                         <br>                            
Best hyperparameters:  {'C': 100, 'l1_ratio': 0.2, 'max_iter': 2000, 'penalty': 'elasticnet', 'solver': 'saga'}<br>         
Best score:  0.6722116939956837                             <br>                                           
------------------------ <br>                                        
Starting Grid Search for Model: LGBMClassifier<br>
Search Finished<br>
Best hyperparameters:  {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}<br>
Best score:  0.61531530223271<br>
------------------------<br>
</font>
<br>
Gradient Boosting and CatBoost were taking too long to figure out so they were dropped

In [11]:
log_reg = LogisticRegression(C=100, l1_ratio=0.2, penalty='elasticnet', solver='saga', max_iter=2000)
log_reg.fit(x_train, y_train)
log_reg.score(x_test, y_test)